In [130]:
import numpy as np
from scipy.stats import norm
import copy

In [159]:
def create_data():


    ##Creating the data -- following the notation in Efron

    genes = 100000
    pi_0 = .90

    #Assuming that the z-scores in the null arm are $N(0,1)$ and $N(arm[1],1)$ in non null case.
    arm = {}
    arm[0] = 0
    arm[1] = -1
    #Confused about: how to do this for two sided alternatives? I.e. something like a t-statistic

    null_or_alt = np.random.binomial(1,pi_0, genes)
    z_values = [ np.random.normal(arm[x]) for x in null_or_alt]
    p_values = [ norm.cdf(z) for z in z_values]
    #np.mean(null_or_alt)   #This is the proportion of genes for which you *should* reject the null
    return p_values, null_or_alt

In [162]:
def Benjamini_Hochberg(p_values, q):
    #
    N = len(p_values)
    sorted_p_values = copy.deepcopy(p_values)
    sorted_p_values = sorted(sorted_p_values)
    i_max = -1
    threshold_p_value = 0
    for j in range(N):
        if sorted_p_values[j] <= q*(j+1)/N:
            i_max = j
            threshold_p_value = sorted_p_values[i_max]
    #print(i_max)
    #We've obtained i_max many genes, and we have controled the expected proportion of them for which we have incorrectly rejected the null
    results = []
    for x in p_values:
        if x > threshold_p_value:
            results.append(0)
        else:
            results.append(1)
    return results
    

In [156]:
def false_positive(results, null_or_alt):
    '''
    null or alt contains the true null or alt values.
    results contains the output of the BH procedure
    '''
    N = len(null_or_alt)
    number_rejected = 0
    num_false_discoveries = 0
    for i in range(N):
        if results[i] == 1:
            number_rejected += 1
            if null_or_alt[i] == 0:
                num_false_discoveries += 1
    return (num_false_discoveries, number_rejected, num_false_discoveries / number_rejected)

In [161]:

trials = 100
fdps = []

for i in range(trials):
    p_values, null_or_alt = create_data()
    q = .1
    results = Benjamini_Hochberg(p_values, q)
    #print(false_positive(results, null_or_alt))
    fdps.append(false_positive(results, null_or_alt)[2])

print(np.mean(fdps))
print(np.var(fdps))

6165
5545
5718
5908
5251
5494
5586
6181
6024
5983
5663
5865
6045
5973
5546
5780
5726
5477
5490
5817
6291
6090
5889
5725
5672
5717
5349
6150
5569
6081
5433
5918
6228
5722
5554
5444
5465
5767
5613
5925
5907
5827
5115
5733
5711
5897
5718
5562
5821
5564
5655
5751
5992
5889
5782
5936
5393
5755
5362
6000
5878
5942
5654
5560
5311
5490
5964
5873
5667
5663
5775
5331
5472
5773
5682
5538
5573
5856
5991
5869
5487
5719
5983
5637
6275
5520
5508
6207
6179
5779
5785
5714
5817
5704
5929
5762
5777
5746
5740
5562
0.009939038079118563
1.9919487812401743e-06


In [ ]:
#We know that the expected false discovery proportion is less than q 